In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from datetime import datetime
from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
plt.rc("font", family = 'SimHei')
plt.rc('font', size = 15)

In [ ]:
#读取
#a1 = pd.read_excel('附件1.xlsx', index_col = 1, header = 0)
a2 = pd.read_excel('附件2.xlsx')
a3 = pd.read_excel('附件3.xlsx')
#a4 = pd.read_excel('附件4.xlsx', index_col = 1, header = 0)
print(a2,a3,sep = '\n\n')

In [ ]:
spinach = [102900005118817]
b2_1 = a2['单品编码'].isin(spinach)
b2_2 = a2[b2_1][['销售日期','扫码销售时间','销量(千克)','销售单价(元/千克)','是否打折销售']]
b3_1 = a3['单品编码'].isin(spinach)
b3_2 = a3[b3_1][['日期','批发价格(元/千克)']]

print(b2_2,b3_2,sep = '\n\n')

In [ ]:
b2_2 = b2_2.reset_index(drop=True)
b3_2 = b3_2.reset_index(drop=True)
b2_2 = b2_2[(b2_2['销售日期'].dt.year != 2023) | (b2_2['销售日期'].dt.month != 6)]
b3_2 = b3_2[(b3_2['日期'].dt.year != 2023) | (b3_2['日期'].dt.month != 6)]
b2_2['销售日期'] = pd.to_datetime(b2_2['销售日期'])
b3_2['日期'] = pd.to_datetime(b3_2['日期'])

In [ ]:
b2_2

In [ ]:
b3_2

In [ ]:
b2_2.to_excel('附件2——2.xlsx', index = False)
b3_2.to_excel('附件3——2.xlsx', index = False)

In [ ]:
is_discount = b2_2['是否打折销售'] == '是'

# 使用groupby和transform来查找同一天非打折销售的单价
# 使用first()来确保我们得到的是同一天的第一条非打折销售记录的单价
non_discount_price = b2_2[~is_discount].groupby('销售日期')['销售单价(元/千克)'].first().astype(float)

# 将非打折销售的单价应用到打折销售的记录上
b2_2.loc[is_discount, '销售单价(元/千克)'] = b2_2.loc[is_discount, '销售日期'].map(non_discount_price)

# 确保修改后的单价保持正确的数据类型
b2_2['销售单价(元/千克)'] = b2_2['销售单价(元/千克)'].astype(float)

In [ ]:
b2_2

In [ ]:
daily_sales_agg = b2_2.groupby('销售日期').agg({
    '销量(千克)': 'sum',
    '销售单价(元/千克)': 'mean'
}).reset_index()

In [ ]:
daily_sales_agg

In [ ]:

# 将批发价格按日期拼接到销售数据后面
# 使用left合并，以销售数据为准
daily_sales_agg = pd.merge(daily_sales_agg, b3_2, left_on='销售日期', right_on='日期', how='left')

# 删除多余的日期列
daily_sales_agg.drop('日期', axis=1, inplace=True)

# 填充缺失的批发价格
# 使用ffill方法前向填充
daily_sales_agg['批发价格(元/千克)'] = daily_sales_agg['批发价格(元/千克)'].fillna(method='ffill')

daily_sales_agg


In [ ]:

# 确保销售日期是datetime类型
daily_sales_agg['销售日期'] = pd.to_datetime(daily_sales_agg['销售日期'])

# 按照销售日期进行分组，并对每个分组的销量求和
#daily_sales_sum = b2_2.groupby('销售日期')['销量(千克)'].sum().reset_index()

# 按照销售日期进行分组，并获取每个分组的第一个销售单价（假设每天的单价是相同的）
#daily_price = b2_2.groupby('销售日期')['销售单价(元/千克)'].first().reset_index()

# 绘制日期-销量图
plt.figure(figsize=(12, 6))
plt.plot(daily_sales_agg['销售日期'], daily_sales_agg['销量(千克)'], marker='o', color='b')
plt.title('日期-销量图')
plt.xlabel('日期')
plt.ylabel('销量(千克)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 绘制日期-单价图
plt.figure(figsize=(12, 6))
plt.plot(daily_sales_agg['销售日期'], daily_sales_agg['销售单价(元/千克)'], marker='x', color='r')
plt.title('日期-单价图')
plt.xlabel('日期')
plt.ylabel('销售单价(元/千克)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
df = daily_sales_agg

In [ ]:
# 确保销售日期是datetime类型
df['销售日期'] = pd.to_datetime(df['销售日期'])

# 生成2020.7.1到2023.2.9之间的所有日期
date_range = pd.date_range(start='2020-07-01', end='2023-02-09')

# 创建一个新的DataFrame，包含生成的日期和空值
new_dates_df = pd.DataFrame(date_range, columns=['销售日期'])
new_dates_df['销量(千克)'] = np.nan
new_dates_df['销售单价(元/千克)'] = np.nan
new_dates_df['批发价格(元/千克)'] = np.nan

# 使用merge方法将原始数据与新创建的日期范围合并
combined_df = pd.merge(df, new_dates_df, on='销售日期', how='left')
df = combined_df
df

In [ ]:
date_range = pd.DataFrame(date_range,columns=['销售日期'])
date_range

In [ ]:
# 确保第二个DataFrame中的日期是datetime类型
date_range['销售日期'] = pd.to_datetime(date_range['销售日期'])

# 使用merge方法将两个DataFrame按日期匹配
merged_df = pd.merge(date_range, df, on='销售日期', how='left').iloc[:,:4]
df = merged_df
df.

In [ ]:
# 确保第二个DataFrame中的日期是datetime类型
df['销售日期'] = pd.to_datetime(df['销售日期'])

# 对销量、销售价格和批发价格进行样条插值
df['销量(千克)'] = df['销量(千克)'].interpolate(method='linear')
df['销售单价(元/千克)'] = df['销售单价(元/千克)'].interpolate(method='linear')
df['批发价格(元/千克)'] = df['批发价格(元/千克)'].interpolate(method='linear')

df

In [ ]:
df.to_excel('附件2——3.xlsx', index = False)

In [ ]:
df = pd.read_excel('附件2——3.xlsx')

In [ ]:
df

In [ ]:
# 系数
alpha_ = 0.0004
beta_ = 0.09
theta_ = 0.9
mu_0 = 0.9
lambda_ = 0.667
m = 0.4
n = 0.6
C_F = 300
C_V = 
C_I = 150
D = 
D_T = 
P_T = 





In [ ]:
# 函数
D_1 = D*(1 - (1/(theta_**((m+1)/3)*np.exp((m-1)*(t_2-t_1)*lambda_/3)+1)))
D_2 = D*(1/(theta_**((4-m)/3)*np.exp((m-1)*(t_2-t_1)*lambda_/3)+1))
P_2 = P_1*(theta_**((4-m)/3)*np.exp((1-m)*(t_2-t_1)*lambda_/3)

mu_1 = 1 / (1 + (1 / mu_0 - 1) * np.exp(lambda_ * t_1))
mu_2 = 1 / (1 + (1 / mu_0 - 1) * np.exp(lambda_ * t_2))

I_11 = I_1
I_22 = I_2

Q = lambda x:D_1 if (P_1-C_V)*D_1-C_F-C_I >= 0 and (mu_1 <= 0.4 or I_11 <= D_1) else 0

def f_I_1(I_11, D_1, Q, mu_1):
    if mu_1 < 0.4:
        I_1 = 0
    elif Q > 0:
        I_1 = max(Q - D_1, 0)
    else :
        I_1 = max(I_11-D_1, 0)
    return I_1

def f_I_2(I_22, I_11, D_2, Q, mu_2):
    if mu_2 < 0.2:
        I_2 = 0
    elif Q > 0:
        I_2 = max(I_11 + I_22 - D_2, 0)
    else:
        I_2 = max(I_22 - D_2, 0)
    return I_2

S_1 = min(I_1,D_1)
S_2 = min(I_2,D_2)

# 示性函数
indicator_I_2 = lambda I_2: 1 if I_2 else 0
indicator_Q = lambda Q:1 if Q else 0

# 目标函数
fun = (S_1*P_1 + S_2*P_2)*(1 - beta_ - indicator_I_2*C_P) - indicator_Q*(C_F + Q*C_V) - C_I

In [ ]:
def f_I_1(I_11, D_1, Q, mu_1):
    if mu_1 < 0.4:
        I_1 = 0
    elif Q > 0:
        I_1 = max(Q - D_1, 0)
    else :
        I_1 = max(I_11-D_1, 0)
    return I_1

def f_I_2(I_22, I_11, D_2, Q, mu_2):
    if mu_2 < 0.2:
        I_2 = 0
    elif Q > 0:
        I_2 = max(I_11 + I_22 - D_2, 0)
    else:
        I_2 = max(I_22 - D_2, 0)
    return I_2


def f_mu_1(Q, I_11, t_1):
    global t_1
    if Q > 0:
        t_1=0
        mu_1 = 1 / (1 + (1 / mu_0 - 1) * np.exp(lambda_ * t_1))
    elif I_11 > 0 :
        t_1+=1
        mu_1 = 1 / (1 + (1 / mu_0 - 1) * np.exp(lambda_ * t_1))
    else:
        t_1 = np.inf
        mu_1 = 0
    return mu_1

def f_mu_2(Q, I_22, t_1, t_2):
    global t_1, t_2
    if Q > 0:
        t_2 = t_1 + 1
        mu_2 = 1 / (1 + (1 / mu_0 - 1) * np.exp(lambda_ * t_2))
    elif I_22 > 0:
        t_2 += 1
        mu_2 = 1 / (1 + (1 / mu_0 - 1) * np.exp(lambda_ * t_2))
    else:
        t_2 = np.inf
        mu_2 = 0
    return mu_2

ls_S_1=[]
ls_S_2=[]
ls_Q=[]
ls_ir_I_2=[]
ls_ir_Q=[]
ls_P=[]
ls_D_T_1=[]
ls_D_T_2=[]
I_1 = 0
I_2 = 0
mu_1 = 0
mu_2 = 0
t_1 = np.inf
t_2 = np.inf

for i in range(len(D)):
    mu_11 = mu_1
    mu_22 = mu_2
    I_11 = I_1
    I_22 = I_2

    Q = lambda x:D[i] if (P_1-C_V[i])*D[i]-C_F-C_I >= 0 and (mu_11 <= 0.4 or I_11+I_22 <= D[i]) else 0

    mu_1 = f_mu_1(Q, I_11, t_1)
    mu_2 = f_mu_2(Q, I_22, t_1, t_2)
    
    if t_1 <= np.inf and t_2 <= np.inf:
        D_1 = D[i]*(1 - (1/(theta_**((m+1)/3)*np.exp((m-1)*(t_2-t_1)*lambda_/3)+1)))
        D_2 = D[i]*(1/(theta_**((4-m)/3)*np.exp((m-1)*(t_2-t_1)*lambda_/3)+1))
       #D_T_1 = D_T[i]*(1 - (1/(theta_**((m+1)/3)*np.exp((m-1)*(t_2-t_1)*lambda_/3)+1)))
        #D_T_2 = D_T[i]*(1/(theta_**((4-m)/3)*np.exp((m-1)*(t_2-t_1)*lambda_/3)+1))
    elif t_1 == np.inf and t_2 == np.inf:
        D_1 = 0
        D_2 = 0
        #D_T_1 = 0
        #D_T_2 = 0
    elif t_2 == np.inf:
        D_1 = D
        D_2 = 0
        #D_T_1 = 0
        #D_T_2 = 0
    elif t_1 == np.inf:
        D_1 = 0
        D_2 = D
        #D_T_1 = 0
        #D_T_2 = 0

    I_1 = f_I_1(I_11, D_1, Q, mu_1)
    I_2 = f_I_2(I_22, I_11, D_2, Q, mu_2)

    indicator_I_2 = lambda I_2: 1 if I_2 else 0
    indicator_Q = lambda Q:1 if Q else 0

    S_1 = min(I_1,D_1)
    S_2 = min(I_2,D_2)
    
    ls_S_1.append(S_1)
    ls_S_2.append(S_2)
    ls_Q.append(Q)
    ls_ir_I_2.append(indicator_I_2)
    ls_ir_Q.append(indicator_Q)

for t in range(len(D)):
    # 定义变量
    P = cp.Variable(2,pos=True)
    
    # 定义目标函数
    fun = (ls_S_1[t]*P[0] + ls_S_2[t]*P[1])*(1 - beta_ - ls_ir_I_2[t]*C_P) - ls_ir_Q[t]*(C_F + ls_Q[t]*C_V[t]) - C_I
    objective = cp.Maximize(fun)
    
    # 定义约束条件
    if ls_t_1[t] <= np.inf and ls_t_2[t] <= np.inf:
        constraints = [P[1] <= P[0]*(theta_**((4-m)/3)*np.exp((1-m)*(t_2-t_1)*lambda_/3)]
    elif ls_t_1[t] == np.inf and ls_t_2[t] == np.inf:
        constraints = [P[1]==0,P[0]==0]
    elif ls_t_2[t] == np.inf:
        constraints = [P[1]==0]
    elif ls_t_1[t] == np.inf:
        constraints = [P[0]==0]
    
    # 构建优化问题
    prob = cp.Problem(objective, constraints)
    
    # 求解优化问题
    prob.solve(solver='GLPK_MI')
    
    # 输出结果
    ls_P.append(P.value)

print(ls_P)

#比较利润
R_T = D_T @ P_T
print(R_T)

In [ ]:
import math
x = math.inf

In [ ]:
x

In [ ]:
s1=np.inf

In [ ]:
s1

In [ ]:
x==s1

In [ ]:
x-s1

In [ ]:
max(-x,0)

In [ ]:
np.exp(np.abs(x)) 

In [ ]:
df = pd.read_excel('数据.xlsx')

In [ ]:
df

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 假设df是包含您数据的pandas DataFrame
# 需要提取销量、销售单价、批发价格列作为特征
features = df[['销量(千克)', '销售单价(元/千克)', '批发价格(元/千克)']].values

# 数据标准化
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# 准备LSTM输入数据
# 假设我们要预测最后一个月的数据，每个月大约30天
X, y = scaled_features[:-30], scaled_features[-30:]

# LSTM需要的输入格式是[samples, time steps, features]
# 这里我们假设每个样本是一天的数据，每个样本只包含一个时间步，特征数量为3
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# 构建LSTM模型
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 3)))
model.add(Dense(3))
model.compile(optimizer='adam', loss='mse')

# 训练模型
model.fit(X, y, epochs=300, verbose=0)

# 进行预测
predicted_scaled = model.predict(X)

# 将标准化后的数据转换回原始的比例
predicted = scaler.inverse_transform(predicted_scaled)

# 打印预测结果
print(predicted)


In [ ]:
# 准备LSTM输入数据
# 假设我们要预测最后一个月的数据，每个月大约30天
# 我们将使用前面的数据作为输入，来预测后面的数据
window_size = 30  # 这是我们的时间步长，即我们用来预测下一个时间点的过去时间点的数量
X, y = [], []

for i in range(window_size, len(scaled_features)):
    X.append(scaled_features[i-window_size:i, :])
    y.append(scaled_features[i, :])

X, y = np.array(X), np.array(y)

# LSTM需要的输入格式是[samples, time steps, features]
# 这里我们假设每个样本是一段时间序列，每个样本包含30个时间步，特征数量为3
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

# 构建LSTM模型
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window_size, 3)))
model.add(Dense(3))
model.compile(optimizer='adam', loss='mse')

# 训练模型
model.fit(X, y, epochs=300, verbose=0)

# 进行预测
# 我们需要预测最后一个月的数据，因此我们将使用最后一个月的数据作为输入
predicted_scaled = model.predict(np.array([scaled_features[-window_size:]]))

# 将标准化后的数据转换回原始的比例
predicted = scaler.inverse_transform(predicted_scaled)

# 打印预测结果
print(predicted)
